In [1]:
import os

In [2]:
%cd C:\Users\shres\Documents\deep-learning-mlops\research


C:\Users\shres\Documents\deep-learning-mlops\research


In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\shres\\Documents\\deep-learning-mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.helper import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.helper import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-27 13:51:21,165: INFO: helper: YAML file loaded successfully from: config\config.yaml]
[2025-07-27 13:51:21,197: INFO: helper: YAML file loaded successfully from: config\params.yaml]
[2025-07-27 13:51:21,199: INFO: helper: Created directory at: artifacts]
[2025-07-27 13:51:21,201: INFO: helper: Created directory at: artifacts/data_ingestion]
[2025-07-27 13:51:21,203: INFO: 1533896105: Downloading data from https://drive.google.com/file/d/1dqiLgcs2fbSvPiXZtwx7AVGNnepYYnge/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1dqiLgcs2fbSvPiXZtwx7AVGNnepYYnge
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1dqiLgcs2fbSvPiXZtwx7AVGNnepYYnge&confirm=t&uuid=85ee05aa-d453-44b2-a466-6736f723354b
To: C:\Users\shres\Documents\deep-learning-mlops\artifacts\data_ingestion\data.zip
100%|██████████| 2.19G/2.19G [01:33<00:00, 23.5MB/s]

[2025-07-27 13:52:56,780: INFO: 1533896105: Downloaded data from https://drive.google.com/file/d/1dqiLgcs2fbSvPiXZtwx7AVGNnepYYnge/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
